In [1]:
#矩阵分解R=PQ，推荐概率模型MCMC采样-似然函数是正态
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

#1.数据集处理
#http://files.grouplens.org/datasets/movielens/ml-20m-README.html
#the following format of file ratings.csv: userId,movieId,rating,timestamp
#The lines within this file are ordered first by userId, then, within user, by movieId.
#Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
data = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False, iterator =True)
data = data.get_chunk(100)
#将userId和movieId全部标准编号
data_rating = data[['rating']]
le = LabelEncoder()
data = data[['userId','movieId']].apply(le.fit_transform)
data = pd.concat([data,data_rating],axis=1)
#抽样10%比例测试
test = data.sample(frac=0.1)
#2.构建概率模型
#概率模型参数设置
uNum = len(data['userId'].unique())#统计用户数
iNum = len(data['movieId'].unique())#统计电影数
mean= data['rating'].max()/2 #正态分布的均值和方差
k = 100 #隐因子数
X_input = theano.shared(data[['userId','movieId']].values)#转numpy array
Y_output = theano.shared(data['rating'].values)#转numpy array
with pm.Model() as BMF_model:
    # Creating the model
    P = pm.Normal('P', mu=mean, sd=mean, shape=(uNum,k))
    Q = pm.Normal('Q', mu=mean, sd=mean, shape=(k,iNum))
    R = pm.Deterministic('R', tt.dot(P,Q))
    rY = []
    for row in X_input.get_value(): # 获取每行的值
        rr = R[int(row[0])][int(row[1])]#userId是0列,movieId是1列
        rY.append(rr)
    Y = pm.Normal('Y',mu=rY, sd=mean, observed=Y_output.get_value())
#3.后验分布计算  
with BMF_model:        
    start=pm.find_MAP()  # 参数初猜
    #二值变量：指定 BinaryMetropolis  离散变量：指定 Metropolis  连续变量：指定 NUTS
    step = pm.Metropolis()
    trace = pm.sample(1000,start=start,step=step,chains=2,cores=8)

#后验分布采样观察
#pm.traceplot(trace, varnames=['P'])
#pm.summary(trace, varnames=['P'])
#print (trace['P'].shape)
#print (trace['Q'].shape)
#4.后验预测  
#X_input.set_value(test[['userId','movieId']].values)#转numpy array
#Y_output.set_value(test['rating'].values)
with BMF_model:
    ppc = pm.sample_posterior_predictive(trace)#vars=BMF_model.observed_RVs
    pred = ppc['Y'].mean(axis=0)
    
print ('RMSE：%f'% mean_squared_error(Y_output.get_value(),pred))

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
INFO (theano.gof.compilelock): Waiting for existing lock by process '15568' (I am process '16185')
INFO (theano.gof.compilelock): To manually release the lock, delete /root/.theano/compiledir_Linux-4.4--generic-x86_64-with-Ubuntu-16.04-xenial-x86_64-3.6.6-64/lock_dir
/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -17,695, ||grad|| = 0.16015: 100%|██████████| 16/16 [00:00<00:00, 219.90it/s]  
Multiprocess sampling (2 chains in 8 jobs)
CompoundStep
>Metropolis: [Q]
>Metropolis: [P]
Sampling 2 chains: 100%|██████████| 3000/3000 [00:06<00:00, 444.96draws/s]

RMSE：0.627957


In [2]:
#矩阵分解R=PQ，推荐概率模型ADVI变分推断-似然函数是正态
import theano
import pymc3 as pm
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error
import theano.tensor as tt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

#1.数据集处理
#http://files.grouplens.org/datasets/movielens/ml-20m-README.html
#the following format of file ratings.csv: userId,movieId,rating,timestamp
#The lines within this file are ordered first by userId, then, within user, by movieId.
#Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).
#Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.
data = pd.read_csv("/data/fjsdata/BayesianRS/ml-20m/ratings.csv",sep=',',low_memory=False, iterator =True)
data = data.get_chunk(100)
#将userId和movieId全部标准编号
data_rating = data[['rating']]
le = LabelEncoder()
data = data[['userId','movieId']].apply(le.fit_transform)
data = pd.concat([data,data_rating],axis=1)
#抽样10%比例测试
test = data.sample(frac=0.1)
#2.构建概率模型
#概率模型参数设置
uNum = len(data['userId'].unique())#统计用户数
iNum = len(data['movieId'].unique())#统计电影数
mean= data['rating'].max()/2 #正态分布的均值和方差
k = 100 #隐因子数
X_input = theano.shared(data[['userId','movieId']].values)#转numpy array
Y_output = theano.shared(data['rating'].values)#转numpy array
with pm.Model() as BMF_model:
    # Creating the model
    P = pm.Normal('P', mu=mean, sd=mean, shape=(uNum,k))
    Q = pm.Normal('Q', mu=mean, sd=mean, shape=(k,iNum))
    R = tt.dot(P,Q)
    rY = []
    for row in X_input.get_value(): # 获取每行的值
        rr = R[int(row[0])][int(row[1])]#userId=0,movieId=1
        rY.append(rr)
    Y = pm.Normal('Y',mu=rY, sd=mean, observed=Y_output.get_value())
#3.后验分布计算  
with BMF_model:        
    inference = pm.ADVI()
    approx = pm.fit(n=10000, method=inference)
    trace = approx.sample(draws=5000)
    
#4.后验预测  
#X_input.set_value(test[['userId','movieId']].values)#转numpy array
#Y_output.set_value(test['rating'].values)
with BMF_model:
    ppc = pm.sample_posterior_predictive(trace)
    pred = ppc['Y'].mean(axis=0)
    
print ('RMSE：%f'% mean_squared_error(Y_output.get_value(),pred))

Average Loss = 5,929.2: 100%|██████████| 10000/10000 [00:39<00:00, 254.78it/s]  
Finished [100%]: Average Loss = 5,921.3
100%|██████████| 5000/5000 [00:10<00:00, 481.68it/s]

RMSE：0.150872
